In [ ]:
# 기본적인 부분
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc("font", family="Nanum Gothic")
plt.rcParams["axes.unicode_minus"]=False

# 데이터 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

# 학습 알고리즘
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet

from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report
from scipy.special import expit, softmax

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree

from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

from sklearn.ensemble import RandomForestClassifier

# 서울시 요식업 매출 예측

## 데이터 불러오기 및 결합

### 서울시 상권 추정매출  

In [ ]:
seoul_sales=pd.read_csv("../data/서울시 상권분석서비스(추정매출-상권).csv", encoding="cp949")
seoul_sales.shape

In [ ]:
seoul_sales.info()

In [ ]:
seoul_sales

In [ ]:
seoul_sales_copy=seoul_sales.copy()

In [ ]:
seoul_restaurant_sales = seoul_sales_copy[seoul_sales_copy["서비스_업종_코드"].str.startswith("CS1")]  # 요식업
seoul_restaurant_sales.head()

### 서울시 상권 유동인구

In [ ]:
seoul_street_people=pd.read_csv("../data/서울시 상권분석서비스(길단위인구-상권).csv", encoding="cp949")
seoul_street_people.shape

In [ ]:
seoul_street_people.info()

In [ ]:
seoul_street_people.head()

### 서울시 상권 직장인구

In [ ]:
seoul_working_people=pd.read_csv("../data/서울시 상권분석서비스(직장인구-상권).csv", encoding="cp949")
seoul_working_people.shape

In [ ]:
seoul_working_people.info()

In [ ]:
seoul_working_people.head()

### 데이터 병합
> 서울시 추정매출: 서비스 업종 별로 나누기 -> 상권별로 서비스 업종의 매출관련 정보  
> 서울시 유동인구 및 직장인구: 상권별로 인구관련 정보  

In [ ]:
seoul_restaurant_sales["서비스_업종_코드_명"].unique()

In [ ]:
seoul_restaurant_list = seoul_restaurant_sales["서비스_업종_코드_명"].unique()

results_with_working_population = {}

for restaurant in seoul_restaurant_list:
    temp_df = seoul_restaurant_sales[seoul_restaurant_sales["서비스_업종_코드_명"] == restaurant]
    
    # 평균매출
    mean_sales = temp_df.groupby(["상권_코드_명", "기준_년분기_코드"])[["당월_매출_금액", "월요일_매출_금액", "화요일_매출_금액", "수요일_매출_금액",
                                                            "목요일_매출_금액", "금요일_매출_금액", "토요일_매출_금액", "일요일_매출_금액"]].mean().reset_index()
    mean_sales.rename(columns={"당월_매출_금액": "평균매출"}, inplace=True)
    
    # 유동인구 데이터와 병합
    merged = pd.merge(mean_sales, seoul_street_people, on=["상권_코드_명", "기준_년분기_코드"], how="left")
    # 직장인구 데이터와 병합
    merged_with_working = pd.merge(merged, seoul_working_people, on=["상권_코드_명", "기준_년분기_코드"], how="left")
    
    # 음식점별로 묶기 (한식음식점 데이터프레임, 중식음식점 데이터프레임, ... )
    results_with_working_population[restaurant] = merged_with_working

In [ ]:
results_with_working_population

In [ ]:
keys=results_with_working_population.keys()
keys

In [ ]:
values=results_with_working_population.values()
values

In [ ]:
items=results_with_working_population.items()
items

In [ ]:
# 음식점 이름을 인덱스로 설정
for restaurant, df in results_with_working_population.items():
    df["음식점"] = restaurant

seoul_restaurants = pd.concat(results_with_working_population.values(), ignore_index=True)
seoul_restaurants

### 1차 컬럼 정리

In [ ]:
seoul_restaurants.columns

In [ ]:
seoul_restaurants["초년_유동인구_수"] = (seoul_restaurants["연령대_10_유동인구_수"] + seoul_restaurants["연령대_20_유동인구_수"])
seoul_restaurants["중년_유동인구_수"] = (seoul_restaurants["연령대_30_유동인구_수"] + seoul_restaurants["연령대_40_유동인구_수"])
seoul_restaurants["노년_유동인구_수"] = (seoul_restaurants["연령대_50_유동인구_수"] + seoul_restaurants["연령대_60_이상_유동인구_수"])

seoul_restaurants["초년_직장_인구_수"] = (seoul_restaurants["연령대_10_직장_인구_수"] + seoul_restaurants["연령대_20_직장_인구_수"])
seoul_restaurants["중년_직장_인구_수"] = (seoul_restaurants["연령대_30_직장_인구_수"] + seoul_restaurants["연령대_40_직장_인구_수"])
seoul_restaurants["노년_직장_인구_수"] = (seoul_restaurants["연령대_50_직장_인구_수"] + seoul_restaurants["연령대_60_이상_직장_인구_수"])

columns_to_drop=["상권_구분_코드_y", "상권_구분_코드_명_y", "상권_코드_y", 
                 "상권_구분_코드_x", "상권_코드_x", 
                 "월요일_유동인구_수", "화요일_유동인구_수", "수요일_유동인구_수", "목요일_유동인구_수", 
                 "금요일_유동인구_수", "토요일_유동인구_수", "일요일_유동인구_수", 
                 "연령대_10_유동인구_수", "연령대_20_유동인구_수", "연령대_30_유동인구_수", 
                 "연령대_40_유동인구_수", "연령대_50_유동인구_수", "연령대_60_이상_유동인구_수", 
                 "연령대_10_직장_인구_수", "연령대_20_직장_인구_수", "연령대_30_직장_인구_수", 
                 "연령대_40_직장_인구_수", "연령대_50_직장_인구_수", "연령대_60_이상_직장_인구_수", 
                 "시간대_00_06_유동인구_수", "시간대_06_11_유동인구_수", "시간대_11_14_유동인구_수", 
                 "시간대_14_17_유동인구_수", "시간대_17_21_유동인구_수", "시간대_21_24_유동인구_수", 
                 "남성연령대_10_직장_인구_수", "남성연령대_20_직장_인구_수", "남성연령대_30_직장_인구_수", 
                 "남성연령대_40_직장_인구_수", "남성연령대_50_직장_인구_수", "남성연령대_60_이상_직장_인구_수", 
                 "여성연령대_10_직장_인구_수", "여성연령대_20_직장_인구_수", "여성연령대_30_직장_인구_수", 
                 "여성연령대_40_직장_인구_수", "여성연령대_50_직장_인구_수", "여성연령대_60_이상_직장_인구_수"]

seoul_restaurants_drop = seoul_restaurants.drop(columns=columns_to_drop)
seoul_restaurants_drop.columns = seoul_restaurants_drop.columns.str.replace("_x", "")
seoul_restaurants_drop.info()

In [ ]:
missing_rows = seoul_restaurants_drop[seoul_restaurants_drop.isnull().any(axis=1)]
missing_rows

결측치들 확인해보니 유동인구, 직장인구 등이 없는 부분이 있어서 결측치 존재했음  
제거하기  

In [ ]:
final_seoul_restaurants=seoul_restaurants_drop.copy()

In [ ]:
final_seoul_restaurants=final_seoul_restaurants.dropna()
final_seoul_restaurants.info()

> 2차 column 정리

In [ ]:
final_seoul_restaurants.columns

In [ ]:
final_seoul_restaurants = final_seoul_restaurants[["기준_년분기_코드", "상권_구분_코드_명", "상권_코드_명", "음식점", "평균매출",
                                                   "월요일_매출_금액", "화요일_매출_금액", "수요일_매출_금액", "목요일_매출_금액",
                                                   "금요일_매출_금액", "토요일_매출_금액", "일요일_매출_금액",
                                                   "총_유동인구_수", "남성_유동인구_수", "여성_유동인구_수", 
                                                   "초년_유동인구_수", "중년_유동인구_수", "노년_유동인구_수", 
                                                   "총_직장_인구_수", "남성_직장_인구_수", "여성_직장_인구_수",
                                                   "초년_직장_인구_수", "중년_직장_인구_수", "노년_직장_인구_수"]]

In [ ]:
final_seoul_restaurants

In [ ]:
final_seoul_restaurants.info()

음식점 -> 원핫인코딩

In [ ]:
final_seoul_restaurants.columns

In [ ]:
final_seoul_restaurants = pd.get_dummies(final_seoul_restaurants, columns=["음식점"], drop_first=False)
final_seoul_restaurants = pd.get_dummies(final_seoul_restaurants, columns=["상권_구분_코드_명"], drop_first=False)
final_seoul_restaurants.info()

In [ ]:
final_seoul_restaurants_copy=final_seoul_restaurants.copy()

In [ ]:
restaurants_final_numeric = final_seoul_restaurants_copy.select_dtypes(include=["float64", "int64", "bool"])
corr_matrix = restaurants_final_numeric.corr()
corr_matrix

In [ ]:
plt.figure(figsize=(18, 12))
sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
correlation_matrix = corr_matrix.corr()["평균매출"]
correlation_matrix_sorted = correlation_matrix.sort_values(ascending=False)
correlation_matrix_sorted

In [ ]:
final_seoul_restaurants_copy.to_csv("../data/seoul_sales_data.csv", index=False)

## 시각화 및 분석

In [ ]:
# seoul_sales_data.csv 파일을 읽어서 데이터프레임으로 저장
csv_path = "../data/seoul_sales_data.csv"
df_seoul = pd.read_csv(csv_path)

### 주요 변수 간 상관관계 히트맵

In [ ]:
# 평균매출과 유동인구/직장인구/연령대별 인구 간의 상관관계를 분석하고 시각화
# 수치형 변수 중 주요 변수만 추려서
corr = df_seoul[["평균매출", "총_유동인구_수", "총_직장_인구_수", "남성_유동인구_수", "여성_유동인구_수", "초년_유동인구_수", "중년_유동인구_수", "노년_유동인구_수"]].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap="Reds", fmt=".2f", square=True)
plt.title("주요 변수 간 상관관계 히트맵")
plt.show()

- 분석 목적 : 평균매출과 유동/직장/성별/연령별 인구 변수들 간의 관계를 파악하기 위해
- 분석 결과 : 요일별 매출 금액과의 상관관계가 가장 높으며, 총직장인구, 성별 직장인구와도 중간 정도의 양의 상관관계를 보임

### 평균 매출 기준 시각화

#### 요일별 평균 매출
- **분석 목적:** 어떤 요일에 매출이 높은지를 파악하기 위해.
- **분석 결과:** 금~토요일 매출이 높으며, 주말 매출이 평일보다 더 큼.

In [ ]:
# 각 요일별 매출 평균을 계산한 후, 매출이 높은 순서대로 막대그래프로 시각화
weekday_sales = df_seoul[[
    "월요일_매출_금액", "화요일_매출_금액", "수요일_매출_금액",
    "목요일_매출_금액", "금요일_매출_금액", "토요일_매출_금액", "일요일_매출_금액"
]].mean()

# 내림차순 정렬
weekday_sales_sorted = weekday_sales.sort_values(ascending=False)

weekday_sales_sorted.plot(kind="bar")
plt.title("요일별 평균 매출 (내림차순)")
plt.ylabel("매출 금액")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

#### 음식점 유형별 평균 매출
- **분석 목적:** 업종별로 매출이 얼마나 차이 나는지 확인하기 위해.
- **분석 결과:** 한식, 양식, 호프/간이주점 업종의 매출이 상대적으로 높게 나타남.

In [ ]:
# '음식점_'으로 시작하는 컬럼명을 찾아서 각 업종에 해당하는 평균 매출 계산
음식점_컬럼들 = [col for col in df_seoul.columns if col.startswith("음식점_")]

음식점_매출 = {
    col.replace("음식점_", ""): df_seoul.loc[df_seoul[col] == True, "평균매출"].mean()
    for col in 음식점_컬럼들
}

# 내림차순 정렬
정렬된_음식점 = sorted(음식점_매출.items(), key=lambda x: x[1], reverse=True)

sns.barplot(
    x=[x[1] for x in 정렬된_음식점],
    y=[x[0] for x in 정렬된_음식점]
)
plt.title("음식점 유형별 평균 매출 (내림차순)")
plt.xlabel("평균 매출")
plt.ylabel("음식점 유형")
plt.show()

#### 상권 유형별 평균 매출
- **분석 목적:** 상권의 종류에 따라 매출이 어떻게 달라지는지 확인.
- **분석 결과:** 관광특구, 발달상권의 매출이 높고, 골목상권이 가장 낮음.

In [ ]:
# 상권 구분 코드명에 따른 매출 평균을 계산
상권_컬럼들 = [col for col in df_seoul.columns if col.startswith("상권_구분_코드_명_")]

상권_매출 = {
    col.replace("상권_구분_코드_명_", ""): df_seoul.loc[df_seoul[col] == True, "평균매출"].mean()
    for col in 상권_컬럼들
}

# 내림차순 정렬
정렬된_상권 = sorted(상권_매출.items(), key=lambda x: x[1], reverse=True)

sns.barplot(
    x=[x[0] for x in 정렬된_상권],
    y=[x[1] for x in 정렬된_상권]
)
plt.title("상권 유형별 평균 매출 (내림차순)")
plt.ylabel("평균 매출")
plt.xlabel("상권 유형")
plt.show()

#### 유동인구별 평균 매출
- **분석 목적:** 상권에 유입되는 전체 인구 수가 매출에 미치는 영향 확인.
- **분석 결과:** 약한 양의 상관관계가 존재하지만, 완전히 선형적이지는 않음.

In [ ]:
# 총 유동인구 수와 평균 매출 간의 관계를 시각화
plt.scatter(df_seoul["총_유동인구_수"], df_seoul["평균매출"])
plt.title("총 유동인구 vs 평균 매출")
plt.xlabel("총 유동인구 수")
plt.ylabel("평균 매출")
plt.grid(True)
plt.show()

#### 연령대 + 유동인구별 평균 매출
- **분석 목적:** 연령대에 따른 유동인구 수가 매출에 미치는 영향 분석.
- **분석 결과:** 중년층(30대)의 유동인구가 많을수록 매출이 높은 경향이 있음.

In [ ]:
# 연령대별 유동인구(초년, 중년, 노년)와 평균 매출의 관계를 산점도로 시각화
plt.scatter(df_seoul["초년_유동인구_수"], df_seoul["평균매출"], alpha=0.4, label="10대", s=10)
plt.scatter(df_seoul["중년_유동인구_수"], df_seoul["평균매출"], alpha=0.4, label="30대", s=10)
plt.scatter(df_seoul["노년_유동인구_수"], df_seoul["평균매출"], alpha=0.4, label="50대+", s=10)
plt.title("연령대별 유동인구 vs 평균 매출")
plt.xlabel("유동인구 수")
plt.ylabel("평균 매출")
plt.legend()
plt.grid(True)
plt.show()

#### 총 직장인구별 평균 매출 
- **분석 목적:** 직장인 수가 많은 지역이 매출에도 영향을 미치는지 확인.
- **분석 결과:** 유동인구보다 직장인구와 매출 간 상관관계가 조금 더 강하게 나타남.


In [ ]:
# 총 직장인구 수와 평균 매출 간의 관계를 시각화
plt.scatter(df_seoul["총_직장_인구_수"], df_seoul["평균매출"], alpha=0.4)
plt.title("총 직장인구 vs 평균 매출")
plt.xlabel("총 직장인구 수")
plt.ylabel("평균 매출")
plt.grid(True)
plt.show()

#### 성별 유동인구별 평균 매출
- **분석 목적** : 남성/여성 유동인구 수가 매출에 어떤 영향을 주는지 확인하기 위해
- **분석 결과** : 여성 유동인구가 많은 지역에서 평균 매출이 더 높은 경향을 보임

In [ ]:
# 남성과 여성 유동인구 수와 평균 매출 간의 관계를 각각 시각화
plt.scatter(df_seoul["남성_유동인구_수"], df_seoul["평균매출"], alpha=0.4, label="남성")
plt.scatter(df_seoul["여성_유동인구_수"], df_seoul["평균매출"], alpha=0.2, color="hotpink", label="여성")
plt.title("성별 유동인구 vs 평균 매출")
plt.xlabel("유동인구 수")
plt.ylabel("평균 매출")
plt.legend()
plt.grid(True)
plt.show()

### 변수 중심 관계 분석

In [ ]:
print(df_seoul.columns.tolist())

#### 상권별 매출 분포

In [ ]:
# melt로 긴 형식으로 바꿔주기
temp_df = pd.DataFrame()

for col in [c for c in df_seoul.columns if c.startswith("상권_구분_코드_명_")]:
    상권명 = col.replace("상권_구분_코드_명_", "")
    temp = df_seoul[df_seoul[col] == True].copy()
    temp["상권유형"] = 상권명
    temp_df = pd.concat([temp_df, temp])

sns.boxplot(x="상권유형", y="평균매출", data=temp_df)
plt.title("상권 유형별 평균 매출 분포")
plt.xticks(rotation=30)
plt.show()

#### 연령대별 유동인구 평균값
- **분석 목적:** 연령대에 따른 유동인구 수가 매출에 미치는 영향 분석.
- **분석 결과:** 중년층(30대)의 유동인구가 많을수록 매출이 높은 경향이 있음.

In [ ]:
# 연령대별 유동인구 평균값을 구하고, 내림차순으로 막대그래프로 시각화
연령대_유동인구 = {
    "10대": df_seoul["초년_유동인구_수"].mean(),
    "30대": df_seoul["중년_유동인구_수"].mean(),
    "50대+": df_seoul["노년_유동인구_수"].mean()
}

# 내림차순 정렬
정렬된_연령대 = dict(sorted(연령대_유동인구.items(), key=lambda x: x[1], reverse=True))

plt.bar(정렬된_연령대.keys(), 정렬된_연령대.values(), color=["darkorange", "darkgreen", "darkblue"])
plt.title("연령대별 유동인구 평균 (내림차순)")
plt.ylabel("유동인구 수")
plt.show()

#### 성별 유동인구 평균값
- **분석 목적:** 남성/여성 유동인구 수와 매출 간 관계를 확인.
- **분석 결과:** 여성 유동인구가 많은 지역에서 매출이 다소 높은 경향을 보임.

In [ ]:
# 남성과 여성의 유동인구 평균값을 비교하고 시각화
성별_유동인구 = {
    "남성": df_seoul["남성_유동인구_수"].mean(),
    "여성": df_seoul["여성_유동인구_수"].mean()
}

정렬된_성별 = dict(sorted(성별_유동인구.items(), key=lambda x: x[1], reverse=True))

plt.bar(정렬된_성별.keys(), 정렬된_성별.values(), color=["hotpink", "darkblue"])
plt.title("성별 유동인구 평균 (내림차순)")
plt.ylabel("유동인구 수")
plt.show()